<a href="https://colab.research.google.com/github/Abhishek603124/Coding_ML_Alorithms_from_Scratch/blob/main/Coding_ML_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

# Linear Regression

In [ ]:
class Linear_Regression:

  def __init__(self):
    self.m = None
    self.b = None


  def fit(self,x_train,y_train):
    x_train_mean = np.mean(x_train)
    y_train_mean = np.mean(y_train)
    num = 0
    den = 0
    num = np.sum((y_train-y_train_mean)*(x_train-x_train_mean))
    den = np.sum((x_train-x_train_mean)**2)
    if den == 0:
      print("Can't compute slope as the denominator is 0")
      self.m = 0
      self.b = y_train_mean
    else:
      self.m = num / den
      self.b = y_train_mean - self.m * x_train_mean
  def predict(self,x_test):
     return self.m*x_test + self.b

# Logistic Regression

In [ ]:
import numpy as np
def sigmoid(self,x):
  return 1/(1+np.exp(-x))
class LOR():
  def __init__(self,lr,iters):
    self.lr=0.01
    self.iters = 1000
  def fit(self,x_train,y_train):
    n_samples,n_features = x.shape[0]
    self.weights = np.zeros(n_features)
    self.bias = 0

    for i in range(self.iters):
      self.linear_pred = np.dot(x_train,self.weights) + self.bias
      predictions = sigmoid(linear_pred)

    # gradients
      dw = (1/n_samples)*np.dot(x_train.T,(predictions-y_train))
      db = (1/n_samples)*np.sum(predictions-y_train)
    # updates
      self.weights-=self.lr*dw
      self.bias-=self.lr*db

  def predict(self,x_test):
    self.linear_pred = np.dot(x_test,self.weights) + self.bias
    predictions = sigmoid(linear_pred)
    class_predictions = [1 if p > 0.5 else 0 for p in predictions]
    return class_predictions

# K-Nearest Neighbour

In [ ]:
import numpy as np
from collections import Counter

def eucledian_distance(x, x_train):
    return np.sqrt(np.sum((x - x_train) ** 2))

class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return predictions

    def _predict(self, x):
        distances = [eucledian_distance(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_nearest_labels).most_common()
        return most_common[0][0]


# Decision Tree
Little tricky

In [ ]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

# Naive Bayes

In [ ]:
import numpy as np
class Naive_Bayes:

  def fit(self,X,y):
    n_samples,n_features = X.shape
    self._classes = np.unique(y)
    n_classes = len(self._classes)

    # calculating the mean, variance, and the prior for each class
    self._mean = np.zeros((n_classes,n_features),dtype = np.float64)
    self._var = np.zeros((n_classes,n_features),dtype = np.float64)
    self._priors = np.zeros((n_classes),dtype = np.float64)

    for idx , c in enumerate(self._classes):
      X_c = X[y==c]
      self._mean[idx, :] = X_c.mean(axis=0)
      self._var[idx,:] = X_c.var(axis=0)
      self._priors[idx] = X_c.shape[0]/float(n_samples)

  def predict(self,X):
    y_pred = [self._predict(x) for x in X]
    return np.array(y_pred)

  def _predict(self,x):
    posteriors = []

    # calculate posterior probability for each class
    for idx, c in enumerate(self._classes):
      prior = np.log(self._priors[idx])
      posterior = np.sum(np.log(self._pdf(idx,x)))
      posterior += prior
      posteriors.append(posterior)

      # return class with higher posterior
    return self._classes[np.argmax(posteriors)]

  def _pdf(self,class_idx,x):
    mean = self._mean[class_idx]
    var = self._var[class_idx]
    numerator = np.exp(((x - mean) ** 2) / (2 * var))
    denominator = np.sqrt(2 * np.pi * var)
    return numerator / denominator

# SVM(Support Vector Machine)

In [ ]:
import numpy as np
class SVM:

    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape

        y_ = np.where(y <= 0, -1, 1)

        # init weights
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]


    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)

# K Means Clustering

In [ ]:
import numpy as np
class KMeans():
  def __init__(self,K=5,max_iters=100,plot_steps=False):
    self.K=K
    self.max_iters=max_iters
    self.plot_steps=plot_steps
# list of sample indices for each clusters
    self.clusters = [[] for _ in range(self.K)]

    # centers(mean) vectors for each clusters
    self.centroids = []

  def predict(self,X):
    self.X=X
    self.n_samples,self.n_features=X.shape

    # initialize the centroids
    random_sample_indices = np.random.choice(self.n_samples,self.K, replace = False)
    self.centroids = [self.X[idx] for idx in random_sample_indices]

    # optimizing the clusters
    for _ in range(self.max_iters):
      # assign samples to the nearest clusters(create clusters)
      self.clusters = self.create_clusters(self.centroids)

      # calculate new centroids
      centroids_old = self.centroids
      self.centroids = self._get_centroids(self.clusters)
  def create_clusters(self,centroids):
    pass
  def _get_centroids(self,clusters):
    pass
  def _is_converged(self,centroids_old,centroids):
    pass